# A Demonstration of the RSA Cryptosystem

The Rivest&ndash;Shamir&ndash;Adleman ([RSA](https://en.wikipedia.org/wiki/RSA_\(cryptosystem\))) cryptosystem demonstrated here combines secure key&ndash;exchange with digital signature to provide for the _confidentiality_, _integrity_ and _authenticity_ of a private message exchanged between two parties (Alice and Bob) over an insecure channel. Moreover, an adversary (Eve) listening on this channel has no practical way of learning the content of the message, nor any way of tampering with it without the receiving party knowing the message has been corrupted.

The key&ndash;exchange component of this cryptosystem provides for message confidentiality utilizing a [_key&ndash;encapsulation mechanism_](https://en.wikipedia.org/wiki/Key_encapsulation_mechanism) (or _KEM_). This differs from more traditional approaches to key negotiation that utilize a key&ndash;_agreement_ mechanism, such as those based on the [discrete logarithm problem](https://en.wikipedia.org/wiki/Discrete_logarithm) introduced by Diffie and Hellman in their seminal 1976 paper [_New Directions in Cryptography_](https://ee.stanford.edu/~hellman/publications/24.pdf).

The digital signature component of this cryptosystem provides for message integrity and authenticity, ideas that were conceived by Diffie and Hellman in their 1976 paper, but not implemented until Rivest, Shamir and Adleman published [_A Method for Obtaining Digital Signatures and Public&ndash;Key Cryptosystems_](https://people.csail.mit.edu/rivest/Rsapaper.pdf) in 1978.

## The Source Code
The following sequence of function calls to the [rsa](https://github.com/dchampion/crypto/tree/master/src/core/rsa.py) module illustrates a session in which Alice signs and encrypts a private message, and then transmits the message to Bob over an insecure channel. Meanwhile, an adversary (Eve) intercepts the message, but cannot feasibly learn its content, nor tamper with it without Bob detecting the message has been corrupted.

It is assumed that Alice, Bob and Eve all possess their own copy of the [rsa](https://github.com/dchampion/crypto/tree/master/src/core/rsa.py) module; but that only Alice and Bob possess the signing and encryption keys produced by this module to provide the aforementioned protections.

Alice and Bob both import the [rsa](https://github.com/dchampion/crypto/tree/master/src/core/rsa.py) module, and a module [sym](https://github.com/dchampion/crypto/tree/master/src/tests/core/sym.py) that implements a symmetric cipher they have agreed in advance will be used to encrypt and decrypt messages (the [sym](https://github.com/dchampion/crypto/tree/master/src/tests/core/sym.py) cipher used here is for illustrative purposes only, and in a real&ndash;world setting would be replaced with a cryptographically&ndash;strong symmetric cipher such as _3DES_ or _AES_).

In [1]:
import os
os.chdir("../src")

# Alice and Bob (and possibly Eve) import the rsa and sym modules required for the session
from core import rsa
from tests.core import sym

Alice begins by generating an RSA keypair. She does this by calling the `make_key` function in the `rsa` module, which returns an `RSAKey` object. She stores her keypair in `key_a` (the suffix `_a` identifies this keypair as belonging to Alice). Alice transmits her public key `key_a.n` to Bob.

In [2]:
# Alice computes her keys, and transmits her public key (key_a.n) to Bob
key_a = rsa.make_key()

Alice must keep the values `key_a.p` and `key_a.q`&mdash;the prime factors of the public modulus `key_a.n`&mdash;secret. She must also keep `key_a.d3` and `key_a.d5`&mdash;her signing and decryption keys, respectively&mdash;secret.

To get an idea of the size and proportions of these parameters, let's print some of them out.

In [3]:
print(key_a.p)

125652339065675312147022382011679717859872078253674584768506536716950469614266207671801517398541491062849256603397430429996196192545111809351832655870242899438035070299743482149191090197437583801188141121972550165476132715641772549261872566842516342502609464163782077776202159171762636346677523085512225104277


In [4]:
print(key_a.q)

164278645207286763321954036203874887035969703358150724148132253921832168043494584220316245048444903436023224130792725276613726589622892691866369873821905203852536943435824965055754054141743672510460952638346881962438884879933626691380929391751611097798366524726051551674452119144616062015147549682450417810193


In [5]:
print(key_a.n)

20641996028835772958221435286173960123866200272881912275062871459931351845706021829900106259544131656352103494329390380258413785172418908016318568300723524569757341405762256995319797871974147406178693179592292189117078773755639540609063601279353018926765701896275519712066951274767460355621946171726526640100982704907911945862953254532808425900190037972723539166042856378605271823115287018717736698455579561726953651810365876239586588310669469574686836497672410653204018173017850252319125896421396864674359262832414128901610339759692080056620445194711771684328926161924271387306374194824540898713109305113327418495461


Note that `key_a.n`, which is the product of `key_a.p` and `key_a.q`, is twice the length of each individually, and that these are very big numbers.

Next, let's print the bit lengths of `key_a.n` and its prime factors.

In [6]:
print(key_a.get_modulus_size())

2048


In [7]:
print(key_a.p.bit_length())

1024


In [8]:
print(key_a.q.bit_length())

1024


Note here that a decimal number with a bit&ndash;length of 2048 (the default key size if none is specified in the `rsa.make_key` function) has approximately 620 digits, and that a decimal number with a bit&ndash;length of 1024 has approximately 310 digits.

Next, we prove that `key_a.n` is in fact the product of `key_a.p` and `key_a.q`.

In [9]:
print(key_a.p * key_a.q == key_a.n)

True


Now it's Bob's turn to generate a keypair, and transmit his public key `key_b.n` to Alice (to differentiate Bob's keypair from Alice's, we name Bob's `key_b`).

In [10]:
# Bob computes his keypair, and transmits the public key (key_b.n) to Alice
key_b = rsa.make_key()

Now that Alice and Bob have generated their respective keys, and transmitted to each other their public moduli `key_a.n` and `key_b.n`, they can proceed with the secure message exchange.

Let's say Alice wants to send the following message securely to Bob. First, Alice stores the message "Sign and encrypt me!" in the variable $mA$.

In [11]:
# Alice's message to Bob
mA = "Sign and encrypt me!"

Then Alice calls the `key_a.sign` method, passing it the message $mA$. She stores the signature in the variable $sA$.

In [12]:
# Alice signs the message, and transmits the signature (sA) to Bob
sA = key_a.sign(mA)

Next, Alice calls the `key_a.encrypt_key` function, passing it Bob's public key `key_b.n`. This computes a session key that will be used to encrypt the private message using a symmetric cipher (e.g., _3DES_ or _AES_). The function returns the pair $KA$ and $cA$, which are, respectively, the symmetric encryption (or _session_) key, and the ciphertext of its input material; the same material Bob will use to recover the session key on his side of the channel.

In [13]:
# Alice computes a session key, and transmits its ciphertext (cA) to Bob
KA, cA = key_a.encrypt_key(key_b.n)

Alice must keep the session key $KA$ private, but she transmits its ciphertext $cA$ to Bob.

Next, Alice encrypts the private message, calling the `encrypt` function of the symmetric cipher, passing it her symmetric key $KA$ and the plaintext of her message $mA$. The result of this operation is the ciphertext $mAC$.

In [14]:
# Alice encrypts the message, and transmits its ciphertext (mAC) to Bob
mAC = sym.encrypt(KA, mA)

Here we print the ciphertext to demonstrate that, if Eve were to intercept it, she would see nothing but a seemingly random string of bytes.

In [15]:
print(mAC)

b'%\x0b%\x8b\x80s\xd9l\xe6WC\x1d\x84`\x02E\x97f\x16]\xcb\x1f3\xd4\xc5z\xfa\xd8x\xefU\x8b'


Next, Alice transmits the message signature $sA$, the message ciphertext $mAC$ and the ciphertext of the symmetric session key $cA$ to Bob.

In order to decrypt the message, Bob must first recover the session key Alice used to encrypt it. He does this by calling  `key_b.decrypt_key`, passing it the ciphertext of the symmetric key $cA$ that Alice sent to him. Because Alice used Bob's public key `key_b.n` to encrypt her session key $KA$, Bob can recover the session key from $cA$ using his corresponding private key, which is hidden inside `key_b`.

In [16]:
# Bob recovers the session key from its ciphertext
KB = key_b.decrypt_key(cA)

Here we prove that the key Bob decrypted above, $KB$, is the same as the key Alice computed, $KA$.

In [17]:
print(KB == KA)

True


Bob calls `decrypt`, passing it $KB$ and the message ciphertext $mAC$ Alice sent to him, to recover Alice's plaintext message $mA$. He stores the result in $mB$.

In [18]:
# Bob decrypts the message cipertext
mB = sym.decrypt(KB, mAC)

Here we prove that the recovered plaintext indeed matches Alice's original message.

In [19]:
print(mB == mA)

True


If we print $mB$, we see that it matches Alice's message $mA$.

In [20]:
print(mB)

Sign and encrypt me!


In order to complete the session, all that remains is for Bob to verify Alice's signature, and thereby prove that the message has not been altered and, further, that it originated from Alice (or, at least, that it was signed with Alice's private key).

Bob calls the `key_b.verify` method, passing Alice's public modulus `key_a.n`, the decrypted message $mB$, and the sigature sent to him by Alice $sA$.

In [21]:
# Bob verifies the message signature
print(key_b.verify(key_a.n, mB, sA))

True


If the assertion above holds, Bob can be assured that the message "Sign and encrypt me!", even if intercepted by Eve, can be read only to him and Alice, and that the message was not corrupted in any way by Eve.